In [1]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
import branca.colormap as cm

# Đọc dữ liệu bất động sản từ file CSV
data = pd.read_csv('../../Data/cleanedData/cleaned_data.csv')

# Loại bỏ các hàng có giá trị thiếu trong cột 'Quận/Huyện' và 'Mức giá'
heatmap_data = data.dropna(subset=['Quận/Huyện', 'Mức giá (triệu/m²)'])

# Chuyển đổi cột 'Mức giá' thành kiểu số
heatmap_data['Mức giá'] = pd.to_numeric(heatmap_data['Mức giá (triệu/m²)'], errors='coerce')

# Tính giá trung bình theo quận/huyện
avg_price_by_district = heatmap_data.groupby('Quận/Huyện')['Mức giá (triệu/m²)'].mean().reset_index()

# Đọc shapefile của Việt Nam và lọc lấy Hà Nội
vietnam_map = gpd.read_file('../Map/gadm41_VNM_shp')
hanoi_map = vietnam_map[vietnam_map['NAME_1'] == 'Hà Nội']

# Ghép dữ liệu giá trung bình vào bản đồ Hà Nội
hanoi_map = hanoi_map.merge(avg_price_by_district, left_on='NAME_2', right_on='Quận/Huyện', how='left')

# Tạo một bản đồ nền bằng folium
hanoi_center = [21.0285, 105.8542]  # Tọa độ trung tâm của Hà Nội
m = folium.Map(location=hanoi_center, zoom_start=11, tiles= None)

# Tạo colormap
colormap = cm.linear.YlOrRd_09.scale(hanoi_map['Mức giá (triệu/m²)'].min(), hanoi_map['Mức giá (triệu/m²)'].max())

# Chuyển đổi GeoDataFrame thành GeoJson và thêm vào bản đồ
geojson = folium.GeoJson(
    hanoi_map,
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['Mức giá (triệu/m²)']) if feature['properties']['Mức giá (triệu/m²)'] else 'lightblue',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
    },
    tooltip=GeoJsonTooltip(
        fields=['NAME_2', 'Mức giá (triệu/m²)'],
        aliases=['Quận/Huyện', 'Giá trung bình (triệu VND)'],
        localize=True
    ),
    highlight_function=lambda feature: {
        'fillColor': 'yellow',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7
    }
).add_to(m)
# Thêm colormap vào bản đồ
colormap.add_to(m)
# Lưu và hiển thị bản đồ
m.save("hanoi_map.html")


f:\anaconda\envs\BTL\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'gadm41_VNM_shp': 'gadm41_VNM_2' (default), 'gadm41_VNM_3'. Specify layer parameter to avoid this warning.
  result = read_func(


In [15]:
import pandas as pd
import matplotlib.pyplot as plt

# Đường dẫn tệp
month_price_path = '../../Data/priceData/month_price.csv'
quarter_price_path = '../../Data/priceData/quarter_price.csv'
data_path = '../../Data/standardizedData/standardized_data.csv'

In [ ]:
import pandas as pd
import plotly.graph_objs as go

# Đường dẫn tệp
month_price_path = '../../Data/priceData/month_price.csv'
quarter_price_path = '../../Data/priceData/quarter_price.csv'
data_path = '../../Data/standardizedData/standardized_data.csv'

# Đọc dữ liệu
data = pd.read_csv(data_path)
month_price = pd.read_csv(month_price_path)
quarter_price = pd.read_csv(quarter_price_path)

# Chỉ số và mã
index = 15  # Bạn có thể thay đổi chỉ số này để vẽ các điểm dữ liệu khác nhau
code = data.loc[index, 'Mã lịch sử giá']

# Lấy dữ liệu giá tương ứng
if code == 'M':
    price_data = month_price[month_price['index'] == index].dropna(axis=1).drop(columns=['index'])
elif code == 'Q':
    price_data = quarter_price[quarter_price['index'] == index].dropna(axis=1).drop(columns=['index'])
else:
    print("Không có thông tin")
    exit()

# Tách dữ liệu giá thành các cột riêng biệt
price_data = price_data.T
price_data.columns = ['Giá']
price_data[['Thấp nhất', 'Trung bình', 'Cao nhất']] = price_data['Giá'].str.split(expand=True)
price_data.drop(columns=['Giá'], inplace=True)

# Chuyển đổi kiểu dữ liệu sang float
price_data = price_data.astype(float)

# Tạo biểu đồ với Plotly
fig = go.Figure()

# Thêm các đường biểu diễn dữ liệu (chỉ có dấu tròn trên đường "Trung bình")
fig.add_trace(go.Scatter(x=price_data.index, y=price_data['Thấp nhất'], mode='lines', name='Thấp nhất',line=dict(color='#FFFF00')))
fig.add_trace(go.Scatter(x=price_data.index, y=price_data['Trung bình'], mode='lines+markers', name='Trung bình', line=dict(color='#336666')))
fig.add_trace(go.Scatter(x=price_data.index, y=price_data['Cao nhất'], mode='lines', name='Cao nhất', line=dict(color='#CC99FF')))

# Tùy chỉnh biểu đồ
fig.update_layout(
    title='Biến động giá theo thời gian',
    xaxis_title='Thời gian',
    yaxis_title='Giá',
    xaxis_tickangle=-45,
    legend_title="Loại giá",
    template="plotly_white",
    hovermode='x unified',  # Thiết lập hovermode để tạo đường kẻ dọc tại điểm chuột
)

# Lưu biểu đồ dưới dạng HTML
fig.write_html("interactive_chart1.html")



In [ ]:
import pandas as pd
import plotly.graph_objs as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Đường dẫn tệp
month_price_path = '../../Data/priceData/month_price.csv'
quarter_price_path = '../../Data/priceData/quarter_price.csv'
data_path = '../../Data/standardizedData/standardized_data.csv'

# Đọc dữ liệu
data = pd.read_csv(data_path)
month_price = pd.read_csv(month_price_path)
quarter_price = pd.read_csv(quarter_price_path)

# Khởi tạo ứng dụng Dash
app = dash.Dash(__name__)

# Layout của ứng dụng
app.layout = html.Div([
    html.H1("Biểu đồ Biến động giá theo thời gian"),

    # Dropdown để chọn index
    dcc.Dropdown(
        id='index-dropdown',
        options=[{'label': f'Index {i}', 'value': i} for i in range(len(data))],
        value=15,  # Giá trị mặc định
         style={
        'width': '200px',
        'font-family': 'sans-serif',   # Kiểu chữ
        'font-size': '12px',                  # Kích thước chữ
        'border-radius': '8px',               # Bo tròn góc
        'background-color': '#CCCCCC',        # Màu nền
        'color': 'black'                       # Màu chữ
        }
    ),

    # Nút để lưu biểu đồ
    html.Button("Lưu thành HTML", id="save-button", n_clicks=0),
    html.Div(id="save-message", style={"margin-top": "10px"}),  # Thông báo lưu thành công

    # Biểu đồ Plotly
    dcc.Graph(id='price-graph')
])

# Callback để cập nhật biểu đồ khi chọn index
@app.callback(
    Output('price-graph', 'figure'),
    [Input('index-dropdown', 'value')]
)
def update_graph(selected_index):
    code = data.loc[selected_index, 'Mã lịch sử giá']
    
    # Lấy dữ liệu giá tương ứng
    if code == 'M':
        price_data = month_price[month_price['index'] == selected_index].dropna(axis=1).drop(columns=['index'])
    elif code == 'Q':
        price_data = quarter_price[quarter_price['index'] == selected_index].dropna(axis=1).drop(columns=['index'])
    else:
        return go.Figure()

    # Tách dữ liệu giá thành các cột riêng biệt
    price_data = price_data.T
    price_data.columns = ['Giá']
    price_data[['Thấp nhất', 'Trung bình', 'Cao nhất']] = price_data['Giá'].str.split(expand=True)
    price_data.drop(columns=['Giá'], inplace=True)

    # Chuyển đổi kiểu dữ liệu sang float
    price_data = price_data.astype(float)

    # Tạo biểu đồ với Plotly
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=price_data.index, y=price_data['Thấp nhất'], mode='lines', name='Thấp nhất', line=dict(color='#FFFF00')))
    fig.add_trace(go.Scatter(x=price_data.index, y=price_data['Trung bình'], mode='lines+markers', name='Trung bình', line=dict(color='#336666')))
    fig.add_trace(go.Scatter(x=price_data.index, y=price_data['Cao nhất'], mode='lines', name='Cao nhất', line=dict(color='#CC99FF')))

    # Tùy chỉnh biểu đồ
    fig.update_layout(
        title='Biến động giá theo thời gian',
        xaxis_title='Thời gian',
        xaxis_tickangle=-45,
        legend_title="Loại giá",
        template="plotly_white",
        hovermode='x unified',  # Thiết lập hovermode để tạo đường kẻ dọc tại điểm chuột
    )
    return fig

# Callback để lưu biểu đồ thành HTML
@app.callback(
    Output("save-message", "children"),
    [Input("save-button", "n_clicks"), Input("index-dropdown", "value")]
)
def save_chart(n_clicks, selected_index):
    if n_clicks > 0:
        # Tạo lại biểu đồ để lưu
        fig = update_graph(selected_index)
        filename = "interactive_chart1.html"  # Tên file cố định
        fig.write_html(filename)
        return f"Biểu đồ đã được lưu thành file {filename}"
    return ""

# Chạy ứng dụng
if __name__ == '__main__':
    app.run_server(debug=True)


In [4]:
# Kết hợp hai biểu đồ vào một trang HTML với liên kết đến các tệp gốc
with open("interactive_chart1.html", "r") as plotly_file, open("hanoi_map.html", "r") as folium_file:
    plotly_html = plotly_file.read()
    folium_html = folium_file.read()

combined_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Kết hợp biểu đồ</title>
</head>
<body>

<h1><a href="interactive_chart1.html" target="_blank">Biểu đồ Biến động giá</a></h1>
<iframe srcdoc='{plotly_html}' width="100%" height="600"></iframe>

<h1><a href="hanoi_map.html" target="_blank">Bản đồ giá bất động sản Hà Nội</a></h1>
<iframe srcdoc='{folium_html}' width="100%" height="600"></iframe>

</body>
</html>
"""

with open("combined_charts.html", "w") as combined_file:
    combined_file.write(combined_html)
